<a href="https://colab.research.google.com/github/parushub/Malayalam-English-Offensive-language-detection/blob/main/Copy_of_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This code is for HOS using multilingual Embeddings for three Dravidian CodeMix languages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Packages to be installed

In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


## Import the required packages

In [ ]:
# packages
import pandas as pd
from collections import Counter
from sentence_transformers import SentenceTransformer
import numpy as np
import sklearn
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)

### Read the data

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/reduced dataset/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/reduced dataset/valid.csv",header=None)

In [ ]:
train['0'] = train['sentence'] + '\t' + train['label'].astype(str)
train[['0']].to_csv('train_formatted.csv', index=False)
test[['0']].to_csv('test_formatted.csv', index=False)


KeyError: 'sentence'

### Sperate the train and test senetnecs and labels to a list

In [ ]:
train_sent_m=[]
train_label_m=[]
test_sent_m = []
test_label_m = []
print(train.columns)
print(test.columns)
for i in train[0]:
    train_sent_m.append(i.split('\t')[0])
    train_label_m.append(i.split('\t')[1])

for i in test[0]:
    test_sent_m.append(i.split('\t')[0])
    test_label_m.append(i.split('\t')[1])

Index([0, 1], dtype='int64')
Index([0, 1], dtype='int64')


IndexError: list index out of range

In [ ]:
print(f"Train sentences: {len(train_sent_m)}, Train labels: {len(train_label_m)}")
print(f"Test sentences: {len(test_sent_m)}, Test labels: {len(test_label_m)}")


Train sentences: 0, Train labels: 0
Test sentences: 0, Test labels: 0


### Label Encoding

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_labels_encoded = le.fit_transform(train_label_m)
dev_labels_encoded = le.fit_transform(test_label_m)

### Get Embeddings

In [ ]:
trans_model = SentenceTransformer('bert-base-multilingual-cased')
# here other multilingual embeddings can be loaded

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
train_sentence_embeddings = trans_model.encode(train_sent_m)
dev_sentence_embeddings = trans_model.encode(test_sent_m)

### Weight calculation

In [ ]:
# class_weights = sklearn.utils.class_weight.compute_class_weight('balanced',np.unique(train_label_m),train_label_m)
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Convert labels to a NumPy array if they are not already
train_label_m = np.array(train_label_m)

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',  # Strategy
    classes=np.unique(train_label_m),  # Unique class labels
    y=train_label_m  # Target labels
)


print(class_weights)

[1.10180696 1.10180696 1.10180696 ... 1.10180696 1.10180696 1.10180696]


### Classification

### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Debugging: Ensure proper encoding of train and test labels
unique_labels_train = set(train_label_m)
unique_labels_test = set(test_label_m)
unique_labels = list(unique_labels_train | unique_labels_test)

le = LabelEncoder()
le.fit(unique_labels)  # Fit the encoder with all unique labels

train_labels_encoded = le.transform(train_label_m)
dev_labels_encoded = le.transform(test_label_m)

# Debugging: Print label counts
print(f"Number of unique labels in training set: {len(unique_labels_train)}")
print(f"Number of unique labels in test set: {len(unique_labels_test)}")
print(f"Total unique labels after encoding: {len(le.classes_)}")

# Logistic Regression model with class weights
class_weight = {i: 1 for i in range(len(le.classes_))}  # Dynamic class weights
model = LogisticRegression(class_weight=class_weight, max_iter=10000)
model.fit(train_sentence_embeddings, train_labels_encoded)

# Make predictions
predicted = model.predict(dev_sentence_embeddings)

# Evaluation scores
print("Evaluation Scores:")
accuracy = accuracy_score(dev_labels_encoded, predicted)
recall = recall_score(dev_labels_encoded, predicted, average="macro", zero_division=1)
precision = precision_score(dev_labels_encoded, predicted, average="macro", zero_division=1)
f1 = f1_score(dev_labels_encoded, predicted, average="macro", zero_division=1)

print("Macro Metrics:")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

# Adjust classification report
# Filter labels present in the test set
test_labels_in_classes = sorted(set(dev_labels_encoded))
target_names = le.inverse_transform(test_labels_in_classes)  # Only use labels present in the test set

print("\nClassification Report:")
print(classification_report(dev_labels_encoded, predicted, labels=test_labels_in_classes, target_names=target_names, zero_division=1))

# Save predictions
predictions = list(le.inverse_transform(predicted))  # Decode predictions back to original labels
classified_df = pd.DataFrame({'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('Logistic_regression.csv', index=False)
print("Predictions saved to 'Logistic_regression.csv'")


Number of unique labels in training set: 4538
Number of unique labels in test set: 980
Total unique labels after encoding: 5356
Evaluation Scores:
Macro Metrics:
Accuracy: 0.165
Precision: 0.587
Recall: 0.505
F1 Score: 0.094

Classification Report:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

### Confusion Matrix

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd

# Assuming 'expected' and 'predicted' are your true and predicted labels
cfm = confusion_matrix(expected, predicted)

# Get the unique classes from the actual and predicted labels
classes = np.unique(np.concatenate((expected, predicted)))

# Convert confusion matrix to a DataFrame for better visualization
df_cfm = pd.DataFrame(cfm, index=classes, columns=classes)

# Plot confusion matrix using seaborn heatmap
plt.figure(figsize=(7, 5))
cfm_plot = sn.heatmap(df_cfm, annot=True, fmt='g', cmap='Blues')

# Save the plot as a PNG file
cfm_plot.figure.savefig("cfm_LR.png")

# Show the plot
plt.show()


NameError: name 'expected' is not defined

### Naive Bayse

In [ ]:
# NAive Baise
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
model = GaussianNB()
model.fit(train_sentence_embeddings, train_labels_encoded)


# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)


print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")

print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('Naive_baise.csv')
print("prediction saved")

## Confusion matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
cfm= confusion_matrix(expected, predicted)
classes = np.unique(train_label_m)

df_cfm = pd.DataFrame(cfm, index = classes, columns = classes)
plt.figure(figsize = (7,5))
cfm_plot = sn.heatmap(df_cfm, annot=True,  fmt='g')
cfm_plot.figure.savefig("cfm_NB.png")

## Random Forest

In [ ]:
#random forest
from sklearn.ensemble import RandomForestClassifier
class_weight = {0:0.22607331, 1:23.54117647 ,2:13.69505703 ,3:17.07014218 ,4:2.484}
model = RandomForestClassifier(n_estimators=100, class_weight=class_weight)
model = model.fit(train_sentence_embeddings, train_labels_encoded)

# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)


print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")

print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('RF.csv')
print("prediction saved")

## Confusion Matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
cfm= confusion_matrix(expected, predicted)
classes = np.unique(train_label_m)

df_cfm = pd.DataFrame(cfm, index = classes, columns = classes)
plt.figure(figsize = (7,5))
cfm_plot = sn.heatmap(df_cfm, annot=True,  fmt='g')
cfm_plot.figure.savefig("cfm_RF.png")

## SVM RBF

In [ ]:
# SVM rbf
from sklearn.svm import SVC
from sklearn import svm
class_weight = {0:0.22607331, 1:23.54117647 ,2:13.69505703 ,3:17.07014218 ,4:2.484}
model = svm.SVC(kernel='rbf',C = 1000, class_weight =class_weight)
model = model.fit(train_sentence_embeddings, train_labels_encoded)

# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)


print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")

print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('SVM_RBF.csv')
print("prediction saved")


## Confusion Matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
cfm= confusion_matrix(expected, predicted)
classes = np.unique(train_label_m)

df_cfm = pd.DataFrame(cfm, index = classes, columns = classes)
plt.figure(figsize = (7,5))
cfm_plot = sn.heatmap(df_cfm, annot=True,  fmt='g')
cfm_plot.figure.savefig("cfm_SVM_rbf.png")

## SVM Poly

In [ ]:
# SVM poly
from sklearn.svm import SVC
from sklearn import svm
model = svm.SVC(kernel='poly',C = 1000)
model = model.fit(train_sentence_embeddings, train_labels_encoded)

# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)


print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")


print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('SVM_poly_labse.csv')
print("prediction saved")

#Save model
import pickle
# Save the trained model as a pickle string.
pkl_filename = "poly_labse.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)
file.close()

## Confusion Matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
cfm= confusion_matrix(expected, predicted)
classes = np.unique(train_label_m)

df_cfm = pd.DataFrame(cfm, index = classes, columns = classes)
plt.figure(figsize = (7,5))
cfm_plot = sn.heatmap(df_cfm, annot =True,  fmt='g')
cfm_plot.figure.savefig("cfm_SVM_poly_labse.png")

## SVM Linear

In [ ]:
# SVM Linear
from sklearn.svm import SVC
from sklearn import svm
model = svm.SVC(kernel='linear',C = 10)
model = model.fit(train_sentence_embeddings, train_labels_encoded)

# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)


print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")

print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('SVM_linear_labse.csv')
print("prediction saved")

#Save model
import pickle
# Save the trained model as a pickle string.
pkl_filename = "linear_labse.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)
file.close()

## Confusion Matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
cfm= confusion_matrix(expected, predicted)
classes = np.unique(train_label_m)

df_cfm = pd.DataFrame(cfm, index = classes, columns = classes)
plt.figure(figsize = (7,5))
cfm_plot = sn.heatmap(df_cfm, annot =True,  fmt='g')
cfm_plot.figure.savefig("cfm_SVM_linear_labse.png")

## Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100)
model.fit(train_sentence_embeddings, train_labels_encoded)

# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)

print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")

print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('Adaboost.csv')
print("prediction saved")

## Confusion matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
cfm= confusion_matrix(expected, predicted)
classes = np.unique(train_label_m)

df_cfm = pd.DataFrame(cfm, index = classes, columns = classes)
plt.figure(figsize = (7,5))
cfm_plot = sn.heatmap(df_cfm, annot=True,  fmt='g')
cfm_plot.figure.savefig("cfm_adaboost.png")

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(weights = 'distance')
model.fit(train_sentence_embeddings, train_labels_encoded)


# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)


print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")

print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('KNN.csv')
print("prediction saved")


## Confusion Matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
cfm= confusion_matrix(expected, predicted)
classes = np.unique(train_label_m)

df_cfm = pd.DataFrame(cfm, index = classes, columns = classes)
plt.figure(figsize = (7,5))
cfm_plot = sn.heatmap(df_cfm, annot=True,  fmt='g')
cfm_plot.figure.savefig("cfm_KNN.png")

## Decision Tree

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
class_weight = {0:0.22607331, 1:23.54117647 ,2:13.69505703 ,3:17.07014218 ,4:2.484}
model = DecisionTreeClassifier(class_weight=class_weight)
model.fit(train_sentence_embeddings, train_labels_encoded)


# make predictions
expected = dev_labels_encoded
predicted = model.predict(dev_sentence_embeddings)


print("eval scores")
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted , average="macro")
precision = precision_score(expected, predicted , average="macro")
f1 = f1_score(expected, predicted, average="macro")

print("macro")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)



print("Classification report")
import sklearn
from sklearn.metrics import classification_report
sklearn.metrics.classification_report(expected, predicted)
target_names = ['Not_offensive', 'Offensive_Targeted_Insult_Group', 'Offensive_Targeted_Insult_Individual', 'Offensive_Untargetede', 'not-malayalam']
print(classification_report(expected, predicted, target_names=target_names))


#Saving the predictions
import csv
predictions = list(le.inverse_transform(predicted))
classified_df = pd.DataFrame( {'tweets': test_sent_m, 'actual_label': test_label_m, 'predictions': predictions})
classified_df.to_csv('DT.csv')
print("prediction saved")
